https://manual.nexusformat.org/classes/base_classes/index.html#base-class-definitions

In [1]:
%pylab notebook

import h5py
import sidpy
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [16]:
file = h5py.File('example_data/33id_spec_22_2D.hdf5','r')

In [8]:
print(file.visititems())

TypeError: visititems() missing 1 required positional argument: 'func'

In [12]:
print(dict(file['/'].attrs).keys())
sidpy.hdf.hdf_utils.print_tree(file)
print(dict(file.attrs))
for key in file['/']:
    print(key)
    print(dict(file[key].attrs))
    
    


dict_keys(['HDF5_Version', 'SPEC_comments', 'SPEC_date', 'SPEC_epoch', 'SPEC_file', 'SPEC_num_headers', 'SPEC_user', 'default', 'h5py_version', 'numpy_version', 'spec2nexus_version'])
/
├ S22
  ---
  ├ G
    -
    ├ G0
    ├ G1
    ├ G2
  ├ MCA
    ---
    ├ first_saved
    ├ last_saved
    ├ number_saved
    ├ reduction_coef
  ├ Q
  ├ T
  ├ command
  ├ comments
  ├ counting_basis
  ├ data
    ----
    ├ Epoch
    ├ H
    ├ I0
    ├ I00
    ├ I0_1
    ├ K
    ├ Kalpha
    ├ L
    ├ _mca_
    ├ _mca_channel_
    ├ chi
    ├ elastic
    ├ eta
    ├ harmonic
    ├ seconds
    ├ signal
    ├ signal2
  ├ date
  ├ definition
  ├ metadata
    --------
    ├ COUPLE_ID_to_DCM
    ├ D3_Amps
    ├ D3_VDC
    ├ D3_bias
    ├ D3_dark
    ├ D3_gain
    ├ D3_pos
    ├ D3_suppr
    ├ D3_time
    ├ DCM_energy
    ├ DCM_lambda
    ├ DCM_mode
    ├ DCM_omega2
    ├ DCM_theta0
    ├ DCM_thetaEnc
    ├ FB_o2_on
    ├ FB_o2_r
    ├ FB_o2_sp
    ├ HSC1b
    ├ HSC1h
    ├ HSC1h0
    ├ HSC1l
    ├ HSC1r
    ├ 

In [35]:
!pip install nexusformat

In [2]:
root = h5py.File('NXarpes.h5', 'w')

# Create the GROUPS 
 
root.create_group('entry')
root['/entry'].attrs['NX_class'] = 'NXentry'
root['/entry'].attrs['EX_required'] = 'true'

In [3]:
# Create the FIELDS 
 
root['/entry'].create_dataset(name='title', data='SAMPLE-CHAR-DATA', maxshape=None)
root['/entry/title'].attrs['type'] = 'NX_CHAR'
root['/entry/title'].attrs['EX_required'] = 'true'

In [4]:
sidpy.hdf.hdf_utils.print_tree(root)

/
├ entry
  -----
  ├ title


In [5]:
def find_NXgroup(h5_group):
    """
    Uses visit() to find all groups with NX_class attribute

    Parameters
    ----------
    h5_group : :class:`h5py.Group`
        Group to search within for the Dataset

    Returns
    -------
    groups : list
        List of [object] corresponding to groups with attribute NX_class.

    """
    if not isinstance(h5_group, (h5py.File, h5py.Group)):
        raise TypeError('h5_group should be a h5py.File or h5py.Group object')

    # print 'Finding all instances of', ds_name
    groups = []

    def __find_name(name, obj):
        if  isinstance(obj, h5py.Group):
            found_nexus_group = False
            for key in dict(obj.attrs).keys():
                if 'NX_class' in key: 
                    found_nexus_group = True
            if found_nexus_group:
                groups.append(obj)
        return

    h5_group.visititems(__find_name)

    return groups

def find_nexus_class(groups, key):
    nexus_class =  None
    for group in groups:
        if  dict(group.attrs)['NX_class'].decode('UTF-8') == key:
            nexus_class = group
            
    return nexus_class

 

In [6]:
timestamp = "T".join(str( datetime.datetime.now() ).split())
timestamp

'2021-09-10T15:13:02.261816'

In [7]:

def read_nexus(file_name):
    file = h5py.File(file_name,'r')
    groups = find_NXgroup(file)

    if find_nexus_class(groups, 'NXentry') is None:
        print('This is not a NeXus file system')
    
    data_group = find_nexus_class(groups, 'NXdata')
    data_dict = dict(data_group.attrs)
    if 'signal' in data_dict:
        signal = data_dict['signal']
        if isinstance(signal, np.bytes_):
            signal = signal.decode('UTF-8')
        print(data_group[data_dict['signal']])
        data = data_group[data_dict['signal']][()]
    if 'axes' in data_dict:
        for axis in data_dict['axes']:
            if isinstance(axis, np.bytes_):
                axis = axis.decode('UTF-8')
            print(data_group[axis][()])
            print(dict(data_group[axis].attrs))

    dataset = sidpy.Dataset.from_array(data)
    
    for i, 
    dataset.set_dimension(0, sidpy.Dimension(np.arange(dataset.shape[0]), 'x',
                                        units='nm', quantity='Length',
                                        dimension_type='spatial'))
dataset.set_dimension(1, sidpy.Dimension(np.linspace(-2, 2, num=dataset.shape[1], endpoint=True), 'y', 
                                        units='nm', quantity='Length',
                                        dimension_type='spatial'))
dataset.set_dimension(2, sidpy.Dimension(np.sin(np.linspace(0, 2 * np.pi, num=dataset.shape[2])), 'bias',
                                        units='mV', quantity='Voltage',
                                        dimension_type='spectral'))
    
    
    return dataset



dataset = read_nexus('example_data/33id_spec_22_2D.hdf5')
    
print(data_group.name)
print(data_dict['chi_indices'])
print(data_dict)
plt.figure()
plt.imshow(dataset)

SyntaxError: invalid syntax (2981641984.py, line 25)

In [134]:
for key in data_group:
    print(key, ( data_group[key]))

Epoch <HDF5 dataset "Epoch": shape (11, 11), type "<f8">
H <HDF5 dataset "H": shape (11, 11), type "<f8">
I0 <HDF5 dataset "I0": shape (11, 11), type "<f8">
I00 <HDF5 dataset "I00": shape (11, 11), type "<f8">
I0_1 <HDF5 dataset "I0_1": shape (11, 11), type "<f8">
K <HDF5 dataset "K": shape (11, 11), type "<f8">
Kalpha <HDF5 dataset "Kalpha": shape (11, 11), type "<f8">
L <HDF5 dataset "L": shape (11, 11), type "<f8">
_mca_ <HDF5 dataset "_mca_": shape (11, 11, 91), type "<f8">
_mca_channel_ <HDF5 dataset "_mca_channel_": shape (91,), type "<i4">
chi <HDF5 dataset "chi": shape (11,), type "<f8">
elastic <HDF5 dataset "elastic": shape (11, 11), type "<f8">
eta <HDF5 dataset "eta": shape (11,), type "<f8">
harmonic <HDF5 dataset "harmonic": shape (11, 11), type "<f8">
seconds <HDF5 dataset "seconds": shape (11, 11), type "<f8">
signal <HDF5 dataset "signal": shape (11, 11), type "<f8">
signal2 <HDF5 dataset "signal2": shape (11, 11), type "<f8">


In [78]:
print(file, dict(file.attrs).keys())

for group in groups:
    if  dict(group.attrs)['NX_class'] == 'NXentry':
        print ('--- found entry')

<HDF5 file "33id_spec_22_2D.hdf5" (mode r)> dict_keys(['HDF5_Version', 'SPEC_comments', 'SPEC_date', 'SPEC_epoch', 'SPEC_file', 'SPEC_num_headers', 'SPEC_user', 'default', 'h5py_version', 'numpy_version', 'spec2nexus_version'])
--- found entry
